In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from PIL import ImageFont, ImageDraw, Image

seed=124
np.random.seed(seed)
tf.random.set_seed(seed)

font = ImageFont.truetype('fonts/gulim.ttc', 30)
colors = [(245,117,16), (117,245,16), (16,117,245), (255,227,79), (254,218,249), (0,102,51), (96,96,96)]

def prob_viz(res, actions, sentence, input_frame, colors):
    output_frame = input_frame.copy()
    
    # show prob
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
    
    # show text box
    cv2.rectangle(output_frame, (0,0), (640, 40), (245, 117, 16), -1) # 텍스트박스
    
    # show sentence text
    output_frame = Image.fromarray(output_frame)
    draw = ImageDraw.Draw(output_frame)
    draw.text( (3,3) , ' '.join(sentence), font=font, fill= (255,255,255))
    
    # show prob text
    for num, prob in enumerate(res):
        draw.text((0, 60+num*40), actions[num], font=font, fill= (255,255,255))

        
    return np.array(output_frame)
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction. Model : holistic, image를 입력해 스켈레톤을 좌표값을 생성.
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

def draw_styled_landmarks(image, results):
    # image : frame, results : model.predict(image)
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 
    
    # 함수화 : 각 좌표값을 한 ndarray로 concat.
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])


mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

# 0. Test in Real time webcam

In [43]:
# Setting
model_name = './weights/20fps_act7_2.h5'
actions = np.array(['나', '목', '아프다', '병원', '어디', '너', '사랑'])
sequence_length = 20


# 1. New detection variables
sequence = [] # 마지막 30프레임 저장
sentence = [] # 표출어 리스트
threshold = 0.999 # prob 0.8 이상일 때 출력

model = tf.keras.models.load_model(model_name)

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarksq
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-1*sequence_length:]
        
        if len(sequence) == sequence_length:
            res = model.predict(np.expand_dims(sequence, axis=0))[0] # output shape : (1,3)
            print(actions[np.argmax(res)]) # print text
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: # 확률 80% 이상일 때 텍스트 추가.
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]: # 이전 텍스트와 다를 때 새로 추가
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)]) # 첫글자는 그냥 추가

            if len(sentence) > 5: # 문자가 5개를 초과하면 앞 문자열 삭제
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, sentence, image, colors) # prob bar를 image에 얹기.
            
        
        # 텍스트 박스
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
나
<class 'mediapipe.python.solution_base.SolutionOutputs'>
나
<class 'mediapipe.python.solution_base.SolutionOutputs'>
나
<class 'mediapipe.python.solution_base.SolutionOutputs'>
나
<class 'mediapipe.python.solution_base.SolutionOutputs'>
나
<class 'mediapipe.python.solution_base.SolutionOutputs'>
나
<class 'mediapipe.python.solution_base.SolutionOutputs'>
나
<class 'mediapipe.python.solution_base.SolutionOutputs'>
나
<class 'mediapipe.python.solution_base.SolutionOutputs'>
나
<class 'mediapipe.python.solution_base.SolutionOutputs'>
나
<class 'mediapipe.python.solution_base.SolutionOutputs'>
나
<class 'mediapipe.python.solution_base.SolutionOutputs'>
나
<class 'mediapipe.python.solution_base.SolutionOutputs'>
나
<class 'mediapipe.python.solution_base.SolutionOutputs'>
나
<class 'mediapipe.python.solution_base.SolutionOutputs'>
나
<class 'mediapipe.python.solution_base.SolutionOutputs'>
나
<class 'mediapipe.python.solution_base.SolutionOutputs'>

<class 'mediapipe.python.solution_base.SolutionOutputs'>
목
<class 'mediapipe.python.solution_base.SolutionOutputs'>
목
<class 'mediapipe.python.solution_base.SolutionOutputs'>
목
<class 'mediapipe.python.solution_base.SolutionOutputs'>
목
<class 'mediapipe.python.solution_base.SolutionOutputs'>
목
<class 'mediapipe.python.solution_base.SolutionOutputs'>
목
<class 'mediapipe.python.solution_base.SolutionOutputs'>
목
<class 'mediapipe.python.solution_base.SolutionOutputs'>
목
<class 'mediapipe.python.solution_base.SolutionOutputs'>
목
<class 'mediapipe.python.solution_base.SolutionOutputs'>
목
<class 'mediapipe.python.solution_base.SolutionOutputs'>
나
<class 'mediapipe.python.solution_base.SolutionOutputs'>
나
<class 'mediapipe.python.solution_base.SolutionOutputs'>
나
<class 'mediapipe.python.solution_base.SolutionOutputs'>
나
<class 'mediapipe.python.solution_base.SolutionOutputs'>
나
<class 'mediapipe.python.solution_base.SolutionOutputs'>
나
<class 'mediapipe.python.solution_base.SolutionOutputs'>

<class 'mediapipe.python.solution_base.SolutionOutputs'>
아프다
<class 'mediapipe.python.solution_base.SolutionOutputs'>
아프다
<class 'mediapipe.python.solution_base.SolutionOutputs'>
아프다
<class 'mediapipe.python.solution_base.SolutionOutputs'>
아프다
<class 'mediapipe.python.solution_base.SolutionOutputs'>
아프다
<class 'mediapipe.python.solution_base.SolutionOutputs'>
아프다
<class 'mediapipe.python.solution_base.SolutionOutputs'>
아프다
<class 'mediapipe.python.solution_base.SolutionOutputs'>
아프다
<class 'mediapipe.python.solution_base.SolutionOutputs'>
아프다
<class 'mediapipe.python.solution_base.SolutionOutputs'>
아프다
<class 'mediapipe.python.solution_base.SolutionOutputs'>
아프다
<class 'mediapipe.python.solution_base.SolutionOutputs'>
아프다
<class 'mediapipe.python.solution_base.SolutionOutputs'>
아프다
<class 'mediapipe.python.solution_base.SolutionOutputs'>
아프다
<class 'mediapipe.python.solution_base.SolutionOutputs'>
아프다
<class 'mediapipe.python.solution_base.SolutionOutputs'>
아프다
<class 'mediapipe.python

<class 'mediapipe.python.solution_base.SolutionOutputs'>
너
<class 'mediapipe.python.solution_base.SolutionOutputs'>
너
<class 'mediapipe.python.solution_base.SolutionOutputs'>
너
<class 'mediapipe.python.solution_base.SolutionOutputs'>
너
<class 'mediapipe.python.solution_base.SolutionOutputs'>
너
<class 'mediapipe.python.solution_base.SolutionOutputs'>
너
<class 'mediapipe.python.solution_base.SolutionOutputs'>
너
<class 'mediapipe.python.solution_base.SolutionOutputs'>
너
<class 'mediapipe.python.solution_base.SolutionOutputs'>
너
<class 'mediapipe.python.solution_base.SolutionOutputs'>
목
<class 'mediapipe.python.solution_base.SolutionOutputs'>
목
<class 'mediapipe.python.solution_base.SolutionOutputs'>
목
<class 'mediapipe.python.solution_base.SolutionOutputs'>
목
<class 'mediapipe.python.solution_base.SolutionOutputs'>
목
<class 'mediapipe.python.solution_base.SolutionOutputs'>
목
<class 'mediapipe.python.solution_base.SolutionOutputs'>
목
<class 'mediapipe.python.solution_base.SolutionOutputs'>

...

<class 'mediapipe.python.solution_base.SolutionOutputs'>
나
<class 'mediapipe.python.solution_base.SolutionOutputs'>
나
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
병원
<class 'mediapipe.python.solution_base.SolutionOutputs'>
병원
<class 'mediapipe.python.solution_base.Sol

<class 'mediapipe.python.solution_base.SolutionOutputs'>
병원
<class 'mediapipe.python.solution_base.SolutionOutputs'>
병원
<class 'mediapipe.python.solution_base.SolutionOutputs'>
병원
<class 'mediapipe.python.solution_base.SolutionOutputs'>
병원
<class 'mediapipe.python.solution_base.SolutionOutputs'>
병원
<class 'mediapipe.python.solution_base.SolutionOutputs'>
병원
<class 'mediapipe.python.solution_base.SolutionOutputs'>
병원
<class 'mediapipe.python.solution_base.SolutionOutputs'>
병원
<class 'mediapipe.python.solution_base.SolutionOutputs'>
병원
<class 'mediapipe.python.solution_base.SolutionOutputs'>
병원
<class 'mediapipe.python.solution_base.SolutionOutputs'>
병원
<class 'mediapipe.python.solution_base.SolutionOutputs'>
병원
<class 'mediapipe.python.solution_base.SolutionOutputs'>
병원
<class 'mediapipe.python.solution_base.SolutionOutputs'>
병원
<class 'mediapipe.python.solution_base.SolutionOutputs'>
병원
<class 'mediapipe.python.solution_base.SolutionOutputs'>
병원
<class 'mediapipe.python.solution_base.S

사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
병원
<class 'mediapipe.python.solution_base.SolutionOutputs'>
병원
<class 'mediapipe.python.solution_base.SolutionOutputs'>
병원
<class 'mediapipe.python.solution_base.SolutionOutputs'>
병원
<class 'mediapipe.python.solution_base.SolutionOutputs'>
병원
<class 'mediapipe.python.solution_base.SolutionOutputs'>
병원
<class 'mediapipe.python.solution_base.SolutionOutputs'>
병원
<class 'mediapipe.python.solution_base.SolutionOutputs'>
병원
<class 'mediapipe.python.solution_base.SolutionOutputs'>
병원
<class 'mediapipe.python.solution_base.SolutionOutputs'>
병원
<class 'mediapipe.python.solution_bas

<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
병원
<class 'mediapipe.python.solution_base.SolutionOutputs'>
병원
<class 'mediapipe.python.solution_base.SolutionOutputs'>
병원
<class 'mediapipe.python.solution_base.S

<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.S

# 1. Record realtime webcam test

In [49]:
# Setting
model_name = './weights/20fps_act7_2.h5'
actions = np.array(['나', '목', '아프다', '병원', '어디', '너', '사랑'])
sequence_length = 20


# 1. New detection variables
sequence = [] # 마지막 30프레임 저장
sentence = [] # 표출어 리스트
threshold = 0.999 # prob 0.8 이상일 때 출력

model = tf.keras.models.load_model(model_name)

cap = cv2.VideoCapture(0)

w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS)*0.4) # fps 조정으로 속도 조절 가능
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
out = cv2.VideoWriter(os.path.join('모두.avi'), fourcc, fps, (w,h))

# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarksq
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-1*sequence_length:]
        
        if len(sequence) == sequence_length:
            res = model.predict(np.expand_dims(sequence, axis=0))[0] # output shape : (1,3)
            print(actions[np.argmax(res)]) # print text
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: # 확률 80% 이상일 때 텍스트 추가.
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]: # 이전 텍스트와 다를 때 새로 추가
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)]) # 첫글자는 그냥 추가

            if len(sentence) > 5: # 문자가 5개를 초과하면 앞 문자열 삭제
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, sentence, image, colors) # prob bar를 image에 얹기.
            
        
        # 텍스트 박스
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
        out.write(image)
    
    out.release()
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
병원
<class 'mediapipe.python.solution_base.SolutionOutputs'>
병원
<class 'mediapipe.python.solution_base.SolutionOutputs'>
병원
<class 'mediapipe.python.solution_base.SolutionOutputs'>
병원
<class 'mediapipe.python.solution_base.SolutionOutputs'>
병원
<class 'mediapipe.python.solution_base.SolutionOutputs'>
어디
<class 'mediapipe.python.solution_base.SolutionOutputs'>
어디
<class 'mediapipe.python.solution_base.SolutionOutputs'>
어디
<class 'mediapipe.python.solution_base.SolutionOutputs'>
어디
<class 'mediapipe.python.solution_base.SolutionOutputs'>
어디
<class 'mediapipe.python.solution_base.SolutionOutputs'>
어디
<class 'mediapipe.python.solution_base.SolutionOutputs'>
어디
<class 'mediapipe.python.solution_base.SolutionOutputs'>
어디
<class 'mediapipe.python.solution_base.SolutionOutputs'>
어디
<class 'mediapipe.python.solution_base.SolutionOutputs'>
어디
<class 'mediapipe.python.solution_base.SolutionOutputs'>
어디
<class 'mediapipe.python.solution_base.S

<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
사랑
<class 'mediapipe.python.solution_base.SolutionOutputs'>
병원
<class 'mediapipe.python.solution_base.SolutionOutputs'>
병원
<class 'mediapipe.python.solution_base.SolutionOutputs'>
병원
<class 'mediapipe.python.solution_base.SolutionOutputs'>
병원
<class 'mediapipe.python.solution_base.SolutionOutputs'>
병원
